In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Thesis/posts1352_standard_representations.csv')

In [42]:
from ast import literal_eval

token_list = []
labels = []

# for x in df['Topic_Probabilities']:
# for x in df['Topic']:
# for x in df['Representative_Docs']:
# for x in df['PreprocessedToken']:
for x in df['PreprocessedToken']:
    token_list.append(x)

corpus = [token.split() for token in token_list]
#bertopic_words = bertopic_words[:1600]

for y in df['HateSpeech']:
    labels.append(y)

#y_list = y_list[:1600]

In [43]:
# obtain token list that's actually list instead of string
from ast import literal_eval

topic_words = []

for x in token_list:
    topic_words.append((literal_eval(x)))

#sequences = topic_words (for topic probabilities)

In [44]:
# change the list containing a list of elements into list of strings (not for topic probabilities)
topic_strings = []

for i in topic_words:

  temp = ' '.join(i)
  topic_strings.append(temp)

In [45]:
# Initialize the tokenizer (not for topic probabilities)
tokenizer = Tokenizer(num_words=1620)
tokenizer.fit_on_texts(topic_strings)
sequences = tokenizer.texts_to_sequences(topic_strings)

In [46]:
import pickle

# Save the tokenizer to a file
with open('/content/drive/MyDrive/Thesis/posts1352_standard_LSTM_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
# Pad the sequences to have the same length
max_sequence_length = 46
data = pad_sequences(sequences, maxlen=max_sequence_length)
data = np.array(data)
labels = np.array(labels)

In [38]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2)

In [47]:
model = Sequential([
    Embedding(input_dim=1620, output_dim=100, input_length=max_sequence_length),
    LSTM(units=100, return_sequences=True),
    LSTM(units=100),
    Dropout(0.5),
    Dense(units=50, activation='relu'),
    Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

# Print the model summary
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 46, 100)           162000    
                                                                 
 lstm_8 (LSTM)               (None, 46, 100)           80400     
                                                                 
 lstm_9 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dropout_9 (Dropout)         (None, 50)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                

In [48]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
41/41 [==============================] - 13s 173ms/step - loss: 0.6936 - accuracy: 0.5015 - val_loss: 0.6934 - val_accuracy: 0.4938
Epoch 2/10
41/41 [==============================] - 5s 113ms/step - loss: 0.6884 - accuracy: 0.5448 - val_loss: 0.6859 - val_accuracy: 0.5247
Epoch 3/10
41/41 [==============================] - 6s 140ms/step - loss: 0.6809 - accuracy: 0.5818 - val_loss: 0.6862 - val_accuracy: 0.5340
Epoch 4/10
41/41 [==============================] - 6s 157ms/step - loss: 0.6740 - accuracy: 0.5764 - val_loss: 0.6874 - val_accuracy: 0.5401
Epoch 5/10
41/41 [==============================] - 5s 111ms/step - loss: 0.6748 - accuracy: 0.5694 - val_loss: 0.6869 - val_accuracy: 0.5340
Epoch 6/10
41/41 [==============================] - 7s 165ms/step - loss: 0.6714 - accuracy: 0.5795 - val_loss: 0.6923 - val_accuracy: 0.5586
Epoch 7/10
41/41 [==============================] - 5s 131ms/step - loss: 0.6709 - accuracy: 0.5872 - val_loss: 0.6882 - val_accuracy: 0.5586
Epoch

In [32]:
# TOPIC
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 0s 30ms/step - loss: 0.6750 - accuracy: 0.5556
Validation Loss: 0.6749892234802246
Validation Accuracy: 0.5555555820465088


In [22]:
# TOPIC PROBABILITY
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 0s 30ms/step - loss: 0.6830 - accuracy: 0.5463
Validation Loss: 0.6830440759658813
Validation Accuracy: 0.5462962985038757


In [41]:
# REPRESENTATIVE DOCUMENT
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 0s 29ms/step - loss: 0.6880 - accuracy: 0.5586
Validation Loss: 0.6880491971969604
Validation Accuracy: 0.5586419701576233


In [ ]:
# PREPROCESSED TOKEN
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 0s 36ms/step - loss: 1.1712 - accuracy: 0.8025
Validation Loss: 1.1712394952774048
Validation Accuracy: 0.8024691343307495


In [ ]:
model.save('/content/drive/MyDrive/Thesis/posts1352_standard_LSTM_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Thesis/posts1352_standard_LSTM_model.h5')